In [ ]:
import yaml
import psycopg2
import pandas as pd
import traceback
import os

def compare_databases(db1_conn, db2_conn,schemas):
    try:  
        dbname1 = db1_conn.info.dbname
        dbname2 = db2_conn.info.dbname
        results = []  # List to store the results

        if schemas is None:
        # Fetch all schemas
            with db1_conn.cursor() as cursor:
                cursor.execute("SELECT schema_name FROM information_schema.schemata WHERE schema_name <> 'information_schema' AND schema_name NOT LIKE 'pg_%'")
                db1_schemas = {row[0] for row in cursor.fetchall()}
            with db2_conn.cursor() as cursor:
                cursor.execute("SELECT schema_name FROM information_schema.schemata WHERE schema_name <> 'information_schema' AND schema_name NOT LIKE 'pg_%'")
                db2_schemas = {row[0] for row in cursor.fetchall()}
        else :
        # Fetch specific schemas            
            with db1_conn.cursor() as cursor:
                cursor.execute(f"SELECT schema_name FROM information_schema.schemata WHERE schema_name in ({schemas})")
                db1_schemas = {row[0] for row in cursor.fetchall()}
            with db2_conn.cursor() as cursor:
                cursor.execute(f"SELECT schema_name FROM information_schema.schemata WHERE schema_name in ({schemas})")
                db2_schemas = {row[0] for row in cursor.fetchall()}                
                
        missing_schemas_db1 = db2_schemas - db1_schemas
        missing_schemas_db2 = db1_schemas - db2_schemas

        if missing_schemas_db1:
            results.append({
                "Database": dbname1,"Schema":'',"Details": "Schema: "+', '.join(missing_schemas_db1),"Table":"","Issue": "Missing Schemas","Type": "Schema" })
        if missing_schemas_db2:
            results.append({
                "Database": dbname2,"Schema":'',"Details": "Schema: "+', '.join(missing_schemas_db2),"Table":"","Issue": "Missing Schemas","Type": "Schema" })

        # For each common schema, compare tables, views, and routines
        for schema in db1_schemas & db2_schemas:
            # Fetch tables and views from db1
            with db1_conn.cursor() as cursor:
                cursor.execute(f"SELECT table_name, table_type FROM information_schema.tables WHERE table_schema = '{schema}'")
                db1_results = cursor.fetchall()
                db1_tables = {row[0] for row in db1_results if row[1] == 'BASE TABLE'}
                db1_views = {row[0] for row in db1_results if row[1] == 'VIEW'}

            # Fetch tables and views from db2
            with db2_conn.cursor() as cursor:
                cursor.execute(f"SELECT table_name, table_type FROM information_schema.tables WHERE table_schema = '{schema}'")
                db2_results = cursor.fetchall()
                db2_tables = {row[0] for row in db2_results if row[1] == 'BASE TABLE'}
                db2_views = {row[0] for row in db2_results if row[1] == 'VIEW'}
            # Compare tables
            missing_tables_db1 = db2_tables - db1_tables
            missing_tables_db2 = db1_tables - db2_tables

            if missing_tables_db1:
                results.append({
                    "Database": dbname1,"Schema": schema,"Table":'',"Details":"Table: "+', '.join(missing_tables_db1),
                    "Issue": "Missing Tables","Type": "Table"})
            if missing_tables_db2:
                results.append({
                    "Database": dbname2,"Schema": schema,"Table":'',"Details": "Table: "+', '.join(missing_tables_db2),
                     "Issue": "Missing Tables","Type": "Table"})

            # Compare views
            missing_views_db1 = db2_views - db1_views
            missing_views_db2 = db1_views - db2_views

            if missing_views_db1:
                results.append({
                    "Database": dbname1,"Schema":schema,"Table":'',"Details": "View: "+', '.join(missing_views_db1),
                    "Issue": "Missing Views" ,"Type": "View"})
            if missing_views_db2:
                results.append({
                    "Database": dbname2,"Schema":schema,"Table":'', "Details": "View: "+', '.join(missing_views_db2),
                    "Issue": "Missing Views" ,"Type": "View"})


            common_views = db1_views & db2_views

            if common_views:
                # Prepare queries
                v_column_query = """
                    SELECT table_name, column_name || ' ' || udt_name AS column_definition
                    FROM information_schema.columns
                    WHERE table_schema = %s AND table_name IN %s;
                """
                view_def_query = """
                    SELECT table_name, COALESCE(view_definition, '') AS view_definition
                    FROM information_schema.views
                    WHERE table_schema = %s AND table_name IN %s;
                """

                # Initialize data structures
                db1_v_columns, db2_v_columns = {}, {}
                db1_view_defs, db2_view_defs = {}, {}

                # Fetch data from both databases
                for conn, columns_dict, view_defs_dict in [
                    (db1_conn, db1_v_columns, db1_view_defs),
                    (db2_conn, db2_v_columns, db2_view_defs)
                ]:
                    with conn.cursor() as cursor:
                        cursor.execute(v_column_query, (schema, tuple(common_views)))
                        for table_name, column_definition in cursor.fetchall():
                            columns_dict.setdefault(table_name, set()).add(column_definition)
                        cursor.execute(view_def_query, (schema, tuple(common_views)))
                        view_defs_dict.update({row[0]: row[1].strip() for row in cursor.fetchall()})

                # Compare views
                for view in common_views:
                    db1_view_columns = db1_v_columns.get(view, set())
                    db2_view_columns = db2_v_columns.get(view, set())
                    db1_view_def = db1_view_defs.get(view)
                    db2_view_def = db2_view_defs.get(view)

                    # Compare columns
                    missing_view_columns_db1 = db2_view_columns - db1_view_columns
                    missing_view_columns_db2 = db1_view_columns - db2_view_columns
                    if missing_view_columns_db1:
                        results.append({
                            "Database": dbname1, "Schema": schema,"Table":"",
                            "Details": f"Missing Columns in {dbname1}: " + ', '.join(missing_view_columns_db1),
                            "Issue": "Missing View Columns/Data Types", "Type": "View"
                        })
                    if missing_view_columns_db2:
                        results.append({
                            "Database": dbname2, "Schema": schema, "Table":"",
                            "Details": f"Missing Columns in {dbname2}: " + ', '.join(missing_view_columns_db2),
                            "Issue": "Missing View Columns/Data Types", "Type": "View"
                        })

                    # Compare definitions
                    if not db1_view_def or not db2_view_def:
                        missing_db = (
                            f"{dbname1} & {dbname2}" if not db1_view_def and not db2_view_def
                            else dbname1 if not db1_view_def
                            else dbname2
                        )
                        issue = "Missing View Definition in Both Databases" if not db1_view_def and not db2_view_def else "Missing View Definition"
                        results.append({
                            "Database": missing_db, "Schema": schema, "Details": f"View: {view}", "Issue": issue,
                            "Type": "View","Table":""
                        })
                    elif db1_view_def != db2_view_def:
                        results.append({
                            "Database": f"{dbname1} vs {dbname2}", "Schema": schema, "Details": f"View: {view}",
                            "Issue": "Mismatch View Definition", "Type": "View","Table":""
                        })



            # Fetch columns and their attributes from both databases for tables
            def fetch_columns(conn, schema, table):
                with conn.cursor() as cursor:
                    cursor.execute("""
                        SELECT c.column_name,c.data_type,c.character_maximum_length,c.numeric_precision,c.numeric_scale,
                               c.udt_name,CASE WHEN c.is_nullable = 'NO' THEN 'NOT NULL' ELSE 'NULL' END AS is_nullable,
                               c.column_default,co.constraint_name,co.constraint_type,co.check_clause,co.ref_table,co.ref_column
                        FROM information_schema.columns c
                        LEFT JOIN (SELECT COALESCE(kcu.column_name, ccu.column_name) AS column_name,tc.constraint_name,tc.constraint_type,cc.check_clause,
                                   ccu.table_name AS ref_table,STRING_AGG(DISTINCT ccu.column_name, ', ') AS ref_column 
                                   FROM information_schema.table_constraints tc
                                   LEFT JOIN information_schema.key_column_usage kcu  ON tc.constraint_name = kcu.constraint_name
                                   LEFT JOIN information_schema.check_constraints cc  ON tc.constraint_name = cc.constraint_name
                                   LEFT JOIN information_schema.constraint_column_usage ccu ON tc.constraint_name = ccu.constraint_name
                                   WHERE tc.table_schema = %s AND tc.table_name = %s
                                   GROUP BY COALESCE(kcu.column_name, ccu.column_name),tc.constraint_name,tc.constraint_type,cc.check_clause,ccu.table_name
                                   ) co ON c.column_name = co.column_name
                        WHERE c.table_schema = %s AND c.table_name = %s;""", (schema, table, schema, table))
                    columns = cursor.fetchall()
                    columns_dict = {row[0]: row[1:] for row in columns}
                return columns_dict

            # Compare columns for each table
            for table in db1_tables & db2_tables:
                db1_columns_type = fetch_columns(db1_conn, schema, table)
                db2_columns_type = fetch_columns(db2_conn, schema, table)

                db1_columns = set(db1_columns_type.keys())
                db2_columns = set(db2_columns_type.keys())

                # Compare columns
                missing_columns_db1 = db2_columns - db1_columns
                missing_columns_db2 = db1_columns - db2_columns
                if missing_columns_db1:
                    results.append({"Database": dbname1, "Schema": schema, "Table": table, "Issue": "Missing Columns", "Details": "Column: "+', '.join(missing_columns_db1),"Type": "Column"})
                if missing_columns_db2:
                    results.append({"Database": dbname2, "Schema": schema, "Table": table, "Issue": "Missing Columns", "Details": "Column: "+', '.join(missing_columns_db2),"Type": "Column"})

                # Compare column attributes
                int_types = {'smallint', 'integer', 'bigint'}
                for column in db1_columns & db2_columns:
                    cmdtsize1 = cmdtsize2 = None  # Initialize variables
                    if db1_columns_type[column][:5] != db2_columns_type[column][:5]:
                        if db1_columns_type[column][0] in int_types:
                            cmdtsize1 = db1_columns_type[column][0]
                        elif db1_columns_type[column][0] in 'ARRAY':
                            cmdtsize1 = ', '.join(map(str, filter(lambda x: x is not None, db1_columns_type[column][:5])))
                        else:
                            cmdtsize1 = ', '.join(map(str, filter(lambda x: x is not None, db1_columns_type[column][:4])))
                        if db2_columns_type[column][0] in int_types:
                            cmdtsize2 = db2_columns_type[column][0]
                        elif db1_columns_type[column][0] in 'ARRAY':
                            cmdtsize2 = ', '.join(map(str, filter(lambda x: x is not None, db2_columns_type[column][:5])))                        
                        else:
                            cmdtsize2 = ', '.join(map(str, filter(lambda x: x is not None, db2_columns_type[column][:4])))
                    if cmdtsize1 and cmdtsize2:
                        results.append({"Database": f"{dbname1} vs {dbname2}", "Schema": schema, "Table": table, "Issue": "Mismatch Column Data Type/Size", 
                                        "Details": f"Column: {column} | {dbname1}: {cmdtsize1} vs {dbname2}: {cmdtsize2}","Type": "Data Type/Size"})


                    if db1_columns_type[column][5] != db2_columns_type[column][5]:
                        results.append({
                            "Database": f"{dbname1} vs {dbname2}","Schema": schema,"Table": table,"Issue": "Mismatch Not Null Constraint",
                            "Details": f"Column: {column} | {dbname1}: {db1_columns_type[column][5]} vs {dbname2}: {db2_columns_type[column][5]}","Type": "Constraint"
                        })

                    if db1_columns_type[column][6] != db2_columns_type[column][6]:
                        results.append({
                            "Database": f"{dbname1} vs {dbname2}", "Schema": schema,"Table": table,"Issue": "Mismatch Default",
                            "Details": f"Column: {column} | {dbname1}: {db1_columns_type[column][6]} vs {dbname2}: {db2_columns_type[column][6]}","Type": "Constraint"
                        })

                    if db1_columns_type[column][7:9] != db2_columns_type[column][7:9]:
                        results.append({
                            "Database": f"{dbname1} vs {dbname2}","Schema": schema,"Table": table,"Issue": "Mismatch Constraint",
                            "Details": f"Column: {column} | {dbname1} Constraint: {db1_columns_type[column][7]} vs {dbname2} Constraint: {db2_columns_type[column][7]}","Type": "Constraint"
                        })

                    else:
                        constraint_type = db1_columns_type[column][8]
                        if db1_columns_type[column][9] != db2_columns_type[column][9]:
                            if constraint_type == 'CHECK':
                                results.append({
                                    "Database": f"{dbname1} vs {dbname2}","Schema": schema,"Table": table,
                                    "Issue": "Mismatch CHECK Constraint Definition",
                                    "Details": f"Column: {column} |  Check Clause | {dbname1}: {db1_columns_type[column][9]} vs {dbname2}: {db2_columns_type[column][9]}","Type": "Constraint"
                                })

                        elif db1_columns_type[column][10:12] != db2_columns_type[column][10:12]:
                            if constraint_type == 'FOREIGN KEY':
                                results.append({
                                    "Database": f"{dbname1} vs {dbname2}","Schema": schema,"Table": table,
                                    "Issue": "Mismatch FOREIGN KEY Constraint Definition",
                                    "Details": f"Column: {column} | {dbname1} Referenced Table: {db1_columns_type[column][10]} | Referenced Column: {db1_columns_type[column][11]} vs "
                                               f"{dbname2} Referenced Table: {db2_columns_type[column][10]} | Referenced Column: {db2_columns_type[column][11]}","Type": "Constraint"
                                })
                            elif constraint_type == 'UNIQUE':
                                results.append({
                                    "Database": f"{dbname1} vs {dbname2}","Schema": schema,"Table": table,
                                    "Issue": "Mismatch UNIQUE Constraint Definition",
                                    "Details": f"Column: {column} | UNIQUE Column |  {dbname1}:{db1_columns_type[column][11]} vs "
                                               f"{dbname2}: {db2_columns_type[column][11]}","Type": "Constraint"
                                })

            # Fetch triggers
            def fetch_triggers(conn, schema, table_name):
                with conn.cursor() as cursor:
                    cursor.execute(f"""
                    SELECT  trigger_name,action_timing AS trigger_timing,
                    STRING_AGG(event_manipulation, ', ' ORDER BY event_manipulation) AS event_types,action_statement AS trigger_definition
                    FROM information_schema.triggers
                    WHERE trigger_schema = '{schema}' AND event_object_table = '{table_name}'
                    GROUP BY event_object_table, trigger_name, action_timing, action_statement;
                    """)
                    return {
                        row[0]: (row[1], row[2], row[3]) for row in cursor.fetchall()
                    }

            # Compare triggers for common tables
            common_tables = db1_tables & db2_tables

            for table_name in common_tables:
                db1_triggers = fetch_triggers(db1_conn, schema, table_name)
                db2_triggers = fetch_triggers(db2_conn, schema, table_name)

                missing_triggers_db1 = set(db2_triggers.keys()) - set(db1_triggers.keys())
                missing_triggers_db2 = set(db1_triggers.keys()) - set(db2_triggers.keys())

                if missing_triggers_db1:
                    results.append({"Database": dbname1, "Schema": schema, "Table": table_name, "Issue": "Missing Triggers", "Details": "Trigger: "+', '.join(missing_triggers_db1),"Type": "Trigger"})
                if missing_triggers_db2:
                    results.append({"Database": dbname2, "Schema": schema, "Table": table_name, "Issue": "Missing Triggers", "Details": "Trigger: "+', '.join(missing_triggers_db2),"Type": "Trigger"})

                for trigger_name in db1_triggers.keys() & db2_triggers.keys():
                    db1_event, db1_timing, db1_definition = db1_triggers[trigger_name]
                    db2_event, db2_timing, db2_definition = db2_triggers[trigger_name]
                    if db1_definition != db2_definition:
                        results.append({"Database": f"{dbname1} vs {dbname2}", "Schema": schema, "Table": table_name, "Issue": "Mismatch Trigger Definition",
                                        "Details": f"Trigger: {trigger_name} | {dbname1}: {db1_definition} | {dbname2}: {db2_definition}","Type": "Trigger"})
                    if db1_event != db2_event or db1_timing != db2_timing:
                        results.append({"Database": f"{dbname1} vs {dbname2}", "Schema": schema, "Table": table_name, "Issue": "Mismatch Trigger Events/Timing",
                                        "Details": f"Trigger: {trigger_name} | {dbname1}: Events: {db1_event} ,Timing: {db1_timing} ,| {dbname2}: Events: {db2_event} ,Timing: {db2_timing} ","Type": "Trigger"})

            # Fetch functions and procedures and separate them
            def fetch_routines(conn, schema):
                with conn.cursor() as cursor:
                    cursor.execute(f"""
                        SELECT r.routine_name || '(' || COALESCE(STRING_AGG(p.udt_name, ', ' ORDER BY p.ordinal_position), '') || ')' AS routine_signature,
                               r.routine_type,
                               r.routine_definition
                        FROM information_schema.routines AS r
                        LEFT JOIN information_schema.parameters AS p ON r.specific_name = p.specific_name
                        WHERE r.routine_schema = '{schema}'
                        GROUP BY r.routine_name, r.specific_name, r.routine_type, r.routine_definition;
                    """)
                    routines = cursor.fetchall()

                # Separate routines into functions and procedures with their definitions
                functions = {
                    row[0]: (row[2].strip() if row[2] is not None else '') 
                    for row in routines if row[1] == 'FUNCTION'}
                procedures = {
                    row[0]: (row[2].strip() if row[2] is not None else '') 
                    for row in routines if row[1] == 'PROCEDURE'}
                return functions, procedures                    



            # Fetch routines from both databases
            db1_functions, db1_procedures = fetch_routines(db1_conn, schema)
            db2_functions, db2_procedures = fetch_routines(db2_conn, schema)                      

            # Compare functions
            missing_functions_db1 = set(db2_functions) - set(db1_functions)
            missing_functions_db2 = set(db1_functions) - set(db2_functions)

            if missing_functions_db1:
                results.append({
                    "Database": dbname1,"Schema": schema,"Type": "Function","Table":"",
                    "Issue": "Missing Functions","Details": f"Function: {', '.join(missing_functions_db1)}"
                })

            if missing_functions_db2:
                results.append({
                    "Database": dbname2,"Schema": schema,"Type": "Function","Table":"",
                    "Issue": "Missing Functions","Details": f"Function: {', '.join(missing_functions_db2)}"
                })

            # Check for function definition mismatches
            common_functions = set(db1_functions) & set(db2_functions)
            function_mismatches = [func for func in common_functions if db1_functions[func] != db2_functions[func]]

            if function_mismatches:
                results.append({
                    "Database": f"{dbname1} vs {dbname2}","Schema": schema,"Type": "Function","Table":"",
                    "Issue": "Mismatch Function Definition","Details": f"Function: {', '.join(function_mismatches)}"
                })

            # Compare procedures
            missing_procedures_db1 = set(db2_procedures) - set(db1_procedures)
            missing_procedures_db2 = set(db1_procedures) - set(db2_procedures)

            if missing_procedures_db1:
                results.append({
                    "Database": dbname1,"Schema": schema,"Type": "Procedure","Table":"",
                    "Issue": "Missing Procedures","Details": f"Procedure: {', '.join(missing_procedures_db1)}"
                })

            if missing_procedures_db2:
                results.append({
                    "Database": dbname2,"Schema": schema,"Type": "Procedure","Table":"",
                    "Issue": "Missing Procedures","Details": f"Procedure: {', '.join(missing_procedures_db2)}"
                })

            # Check for procedure definition mismatches
            common_procedures = set(db1_procedures) & set(db2_procedures)
            procedure_mismatches = [proc for proc in common_procedures if db1_procedures[proc] != db2_procedures[proc]]

            if procedure_mismatches:
                results.append({
                    "Database": f"{dbname1} vs {dbname2}","Schema": schema,"Type": "Procedure","Table":"",
                    "Issue": "Mismatch Procedure Definition","Details": f"Procedure: {', '.join(procedure_mismatches)}"
                })


            def fetch_sequences(conn, schema):
                with conn.cursor() as cursor:
                    cursor.execute(f"""
                        SELECT sequence_name, data_type
                        FROM information_schema.sequences
                        WHERE sequence_schema = '{schema}'
                    """)
                    return {row[0]: row[1] for row in cursor.fetchall()}

                # Compare sequences for common schemas

            db1_sequences = fetch_sequences(db1_conn, schema)
            db2_sequences = fetch_sequences(db2_conn, schema)

            # Compare sequence names
            common_sequences = set(db1_sequences.keys()) & set(db2_sequences.keys())
            missing_sequences_db1 = set(db2_sequences.keys()) - set(db1_sequences.keys())
            missing_sequences_db2 = set(db1_sequences.keys()) - set(db2_sequences.keys())

            # Report missing sequences
            if missing_sequences_db1:
                results.append({
                    "Database": dbname1,"Schema": schema,"Table":"","Issue": "Missing Sequences","Type":"Sequence",
                    "Details": f"Sequence: {', '.join(missing_sequences_db1)}"
                })

            if missing_sequences_db2:
                results.append({
                    "Database": dbname2,"Schema": schema,"Table":"","Issue": "Missing Sequences","Type":"Sequence",
                    "Details": f"Sequence: {', '.join(missing_sequences_db2)}"
                })

            # Compare data types for common sequences
            for sequence_name in common_sequences:
                db1_data_type = db1_sequences[sequence_name]
                db2_data_type = db2_sequences[sequence_name]

                if db1_data_type != db2_data_type:
                    results.append({
                        "Database": f"{dbname1} vs {dbname2}","Schema": schema,"Table":"","Issue": "Mismatch Sequence Datatype",
                        "Details": f"Sequence: {sequence_name} | {dbname1}: {db1_data_type}, {dbname2}: {db2_data_type},Type: Sequence"
                    })


            def fetch_data_types(conn, schema):
                with conn.cursor() as cursor:
                    cursor.execute(f"""
                        SELECT udt_name,
                        STRING_AGG(attribute_name || ' ' || attribute_udt_name, ', ' ORDER BY ordinal_position) as data_type_field
                        FROM information_schema.attributes 
                        WHERE udt_schema = '{schema}'
                        GROUP BY udt_name
                    """)
                    return {row[0]: set(row[1].split(', ')) for row in cursor.fetchall()}

            # Fetch data types for the schema in both databases
            db1_data_types = fetch_data_types(db1_conn, schema)
            db2_data_types = fetch_data_types(db2_conn, schema)

            # Compare data types for common types
            common_data_types = set(db1_data_types.keys()) & set(db2_data_types.keys())
            missing_data_types_db1 = set(db2_data_types.keys()) - set(db1_data_types.keys())
            missing_data_types_db2 = set(db1_data_types.keys()) - set(db2_data_types.keys())

            # Report missing data types
            if missing_data_types_db1:
                results.append({
                    "Database": dbname1,"Schema": schema,"Table":"","Issue": "Missing Types","Type":"Type",
                    "Details": f"Data Type: {', '.join(missing_data_types_db1)}"
                })

            if missing_data_types_db2:
                results.append({
                    "Database": dbname2,"Schema": schema,"Table":"","Issue": "Missing Types","Type":"type",
                    "Details": f"Data Type: {', '.join(missing_data_types_db2)}"
                })

            # Compare data type fields and report differences
            for data_type_name in common_data_types:
                db1_fields = db1_data_types[data_type_name]
                db2_fields = db2_data_types[data_type_name]

                missing_db1_fields = db2_fields - db1_fields
                missing_db2_fields = db1_fields - db2_fields


                if missing_db1_fields or missing_db2_fields:
                    issue_details = []
                    if missing_db1_fields:
                        issue_details.append(f"{dbname1}: {', '.join(missing_db1_fields)}")
                    if missing_db2_fields:
                        issue_details.append(f"{dbname2}: {', '.join(missing_db2_fields)}")

                    results.append({
						"Database": f"{dbname1} vs {dbname2}",
                        "Schema": schema,"Type": "Type","Table":"",
                        "Issue": "Mismatch Types Fields",
                        "Details": f"Data Type: {data_type_name} | {'| '.join(issue_details)}"
                    })
                    
        # Convert results to DataFrame
        df = pd.DataFrame(results)

        # Remove rows where all columns are empty
        df = df.dropna(how='all')
        # Check if the DataFrame is empty after removing empty rows
        
        if df.empty:
            if db1_schemas&db2_schemas:
                print("No difference in both databases")
            else:
                print("No common schemas exist in both databases.")
        else:
            df = df[['Database', 'Schema', 'Table', 'Details', 'Issue', 'Type']]
            # Define the output path and file name
            username = os.environ['USERNAME']
            output_path = f"C:\\Users\\{username}\\Downloads"
            output_file = f"db_comparison_{dbname1}_{dbname2}_{pd.Timestamp.now().strftime('%Y%m%d%H%M%S')}.xlsx"

            # Export to Excel
            df.to_excel(f"{output_path}\\{output_file}", index=False)
            print(f"Comparison results exported to {output_file}")
                    
    
    except Exception as e:
        # Get the exception traceback details
        tb = traceback.format_exc()
        # Print the exception message and the line number
        print(f"An unexpected error occurred: {str(e)}")
        print(f"Details: {tb}")

if __name__ == "__main__":
    # Load configuration from YAML file
    with open("C:\\Users\\Downloads\\config.yaml", "r") as config_file:
        config = yaml.safe_load(config_file)

    db1_params = config["db1_params"]
    db2_params = config["db2_params"]

    # Connect to databases
    def connect_to_db(params, db_label):
        try:
            conn = psycopg2.connect(**params)
            print(f"Connected to {db_label}: {params['database']} successfully.")
            return conn
        except psycopg2.OperationalError as e:
            print(f"Error connecting to {db_label}: {params['database']}.\nDetails:{e}")
            return None

    conn1 = connect_to_db(db1_params, "Database 1")
    conn2 = connect_to_db(db2_params, "Database 2")

    if conn1 and conn2:
        # Prompt user for input
        choice = input("Do you want to compare the whole database or specific schemas? (Enter 1 for 'whole' or 2 for 'schema'): ").strip().lower()
        if choice == "1":
            print('whole databases comparing...')
            compare_databases(conn1, conn2,schemas=None)
        elif choice == "2":
            schemas_input = input("Enter schemas to compare (comma-separated): ").strip()
            schemas = ", ".join(f"'{schema.strip()}'" for schema in schemas_input.split(","))
            print(f"specific schema:{schemas} databases comparing...")
            compare_databases(conn1, conn2, schemas=schemas)
        else:
            print("Invalid choice. Exiting.")

        # Close connections
        conn1.close()
        conn2.close()
    else:
        print("One or both database connections failed. Skipping comparison.")